**Maestría en Informática - PUCP - Aprendizaje Automático**

**Proyecto Grupal 2022-1**

# **Complicaciones en pacientes que sufrieron infarto al miocardio**

Repositorio: https://github.com/dp202204/Proyecto-ML-2022

In [25]:
import pandas as pd

In [26]:
# Carga de datos - Pacientes que sufrieron infarto al miocardio y sus complicaciones
# Datos utilizados:
# Myocardial infarction complications Data Set
# Center for Machine Learning and Intelligent Systems, Bren School of Information and Computer Science, University of California, Irvine

mic_data1 = pd.read_csv('https://leicester.figshare.com/ndownloader/files/23581310')

In [27]:
mic_data1.shape

(1700, 124)

In [28]:
# Recolección de información general acerca de las columnas y su contenido

cant_col_nan      = 0
cant_col_sin_nan  = 0
cant_col_num      = 0
cant_col_binarias = 0

for col in mic_data1:
  #Determinamos cuantas columna son numericas
  if pd.api.types.is_numeric_dtype(mic_data1[col]):
    cant_col_num = cant_col_num + 1
    if mic_data1[col].nunique() == 2:
      cant_col_binarias = cant_col_binarias + 1
  #Determinamos que columnas tienen datos faltantes
  if mic_data1[col].isna().sum() > 0:
    cant_col_nan = cant_col_nan + 1
  else:
     cant_col_sin_nan = cant_col_sin_nan + 1

In [29]:
cant_col_mic_data1 = mic_data1.shape[1]
print('Total de columnas en el conjunto de datos: ',cant_col_mic_data1)
print('Total de columnas numericas:               ',cant_col_num, ' (', '%.1f%%' % (100*cant_col_num/cant_col_mic_data1),')')
print('Total de columnas numericas binarias:       ',cant_col_binarias, ' (', '%.1f%%' % (100*cant_col_binarias/cant_col_mic_data1),')')
print('Total de columnas con datos faltantes:     ',cant_col_nan, ' (', '%.1f%%' % (100*cant_col_nan/cant_col_mic_data1),')')
print('Total de columnas con datos completos:      ',cant_col_sin_nan, ' (Para comprobación)')

Total de columnas en el conjunto de datos:  124
Total de columnas numericas:                124  ( 100.0% )
Total de columnas numericas binarias:        89  ( 71.8% )
Total de columnas con datos faltantes:      110  ( 88.7% )
Total de columnas con datos completos:       14  (Para comprobación)


Todas las columnas son numéricas. Algunas son enteras y otras de punto flotante.

89 columnas (71.8%) son binarias, es decir, tiene los valores 0 y 1. Por tanto 35 columan tiene datos no binarios.

La mayoría de las columnas tienen datos faltantes (88.7%).

In [30]:
# Recoleccion de características más específicas de los datos por columna: Datos unicos, datos faltantes

cant_filas_mic_data1 = mic_data1.shape[0]
mic_caract = pd.DataFrame(mic_data1.nunique(),columns=['Datos_Unicos'])
mic_caract['Faltantes'] = mic_data1.isna().sum() / cant_filas_mic_data1 * 100
# mic_data.nunique().hist()
# mic_data.nunique()

In [31]:
print ('Columnas con mayor cantidad de datos únicos')
print ()
mic_caract.sort_values('Datos_Unicos', ascending = False)['Datos_Unicos'].head(37)

Columnas con mayor cantidad de datos únicos



ID             1700
L_BLOOD         174
ALT_BLOOD        69
AGE              62
ROE              58
AST_BLOOD        58
K_BLOOD          51
NA_BLOOD         40
S_AD_ORIT        32
S_AD_KBRIG       30
D_AD_KBRIG       21
D_AD_ORIT        20
TIME_B_S          9
LET_IS            8
DLIT_AG           8
STENOK_AN         7
post_im           5
inf_im            5
lat_im            5
ant_im            5
NA_R_1_n          5
NOT_NA_1_n        5
ZSN_A             5
FK_STENOK         5
KFK_BLOOD         4
R_AB_2_n          4
R_AB_1_n          4
NA_R_2_n          4
NOT_NA_2_n        4
INF_ANAM          4
R_AB_3_n          4
GB                4
NOT_NA_3_n        3
NA_R_3_n          3
IBS_POST          3
NA_KB             2
fibr_ter_06       2
Name: Datos_Unicos, dtype: int64

De acuerdo al análisis de datos únicos, solo la columna ID contiene información que no sería útil.

Descartaremos la columna ID.

In [32]:
print ('Columnas con mayor porcentaje de datos faltantes')
print ()
mic_caract.sort_values('Faltantes', ascending = False)['Faltantes'].head(15)

Columnas con mayor porcentaje de datos faltantes



KFK_BLOOD     99.764706
IBS_NASL      95.764706
S_AD_KBRIG    63.294118
D_AD_KBRIG    63.294118
NOT_NA_KB     40.352941
LID_KB        39.823529
NA_KB         38.647059
GIPER_NA      22.058824
NA_BLOOD      22.058824
K_BLOOD       21.823529
GIPO_K        21.705882
AST_BLOOD     16.764706
ALT_BLOOD     16.705882
S_AD_ORIT     15.705882
D_AD_ORIT     15.705882
Name: Faltantes, dtype: float64

De acuerdo a algunos autores, la proporción de valores faltantes en estudios científicos es muy variable y puede alcanzar niveles de entre 26% to 72% para estudios sobre educación y psicología, siendo valores comunes entre 15% to 20%.

(Yiran Dong, Chao-Ying Joanne Peng, Principled missing data methods for researchers, Springerplus. 2013; 2: 222. Published online 2013 May 14. doi: 10.1186/2193-1801-2-222
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3701793/ )

Notas diversas encontradas en foros de internet sugieren valores entre 25% y 30% como aceptables.

Para el presente trabajo descartaremos las columnas con más de 25% de datos faltantes.

Descartaremos 7 columnas, que serán las siguientes:

- KFK_BLOOD (Serum CPK content). Faltantes: 99.8%

- IBS_NASL (Heredity on CHD)). Faltantes: 95.8%

- S_AD_KBRIG (Systolic blood pressure according to Emergency Cardiology Team). Faltantes: 63.3%

- D_AD_KBRIG (Diastolic blood pressure according to Emergency Cardiology Team). Faltantes: 63.3%

- NOT_NA_KB (Use of NSAIDs by the Emergency Cardiology Team). Faltantes: 40.4%

- LID_KB (Use of lidocaine by the Emergency Cardiology Team). Faltantes: 39.8%

- NA_KB (Use of opioid drugs by the Emergency Cardiology Team). Faltantes: 38.6%

Nota: Todas estas columnas corresponden a datos médicos de los pacientes durante su tratamiento. No contienen información sobre las complicaciones.

In [34]:
# PREPROCESAMIENTO - PASO 1
# Eliminación de columnas con muchos datos únicos y faltantes.
# Se eliminan 8 columnas en total.

mic_data2 = mic_data1.copy()

col_elim = ['ID','KFK_BLOOD','IBS_NASL','S_AD_KBRIG','D_AD_KBRIG','NOT_NA_KB','LID_KB','NA_KB']
mic_data2.drop(col_elim, axis=1, inplace=True)

In [40]:
mic_data2.shape

(1700, 116)

De acuerdo a la descripción de datos, las últimnas 12 columnas tienen informacion sobre las complicaciones.
- Columnas 113 a 123 (número original de columna): Complicaciones específicas. Todas tienen valores binarios.
- Columna 124 (número original de columna) - LET_IS: Causa de resultado letal. Contiene un número: Cero significa que el caso no fue letal. Del 1 al 7 indica la causa de la letalidad.

Analizaremos qué proporción de casos presentaron complicaciones, cuántas complicaciones se pueden presentar en cada caso y qué proporción de casos fueron letales.

In [57]:
col_causas = ['FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD','A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN','REC_IM','P_IM_STEN']

tuvo_complic = mic_data2[col_causas].sum(axis=1) > 0

# mic_data2['CON_COMPLIC'] = mic_data2[tuvo_complic]
# mic_data2['LETAL'] = mic_data2['LET_IS'] > 0
mic_data2['NUM_COMPLIC'] = mic_data2[col_causas].sum(axis=1)

print('Casos ordenados por número de complicaciones:')
mic_data2.sort_values('NUM_COMPLIC', ascending = False)['CON_COMPLIC'].head(15)

Casos ordenados por número de complicaciones:


1121    5
1376    5
1163    4
1624    4
1489    4
1374    4
1288    4
1197    4
1194    4
894     4
1226    4
1178    4
1005    4
1022    4
1258    4
Name: CON_COMPLIC, dtype: int64

**Se aprecia que puede haber casos hasta con 5 complicaciones.**

**Por lo tanto, el modelo predictivo no podría usar las columnas de complicaciones como categoría de clasificación porque no son excluyentes.**

In [65]:
seleccion_casos_complic = mic_data2['NUM_COMPLIC'] > 0
num_casos_complic = len(mic_data2.loc[mic_data2['NUM_COMPLIC'] > 0])

num_casos_let = len(mic_data2.loc[mic_data2['LET_IS'] > 0])

cant_casos_mic_data2 = mic_data2.shape[0]
print('Total de casos en el conjunto de datos: ',cant_casos_mic_data2)
print('Total de casos con complicaciones:      ',num_casos_complic, ' (', '%.1f%%' % (100*num_casos_complic/cant_casos_mic_data2),')')
print('Total de casos letales:                 ',num_casos_let, ' (', '%.1f%%' % (100*num_casos_let/cant_casos_mic_data2),')')

Total de casos en el conjunto de datos:  1700
Total de casos con complicaciones:       935  ( 55.0% )
Total de casos letales:                  271  ( 15.9% )


# El modelo podría enfoarse en predecir si habrá complicaciones en general o si habrá complicacaiones letales.

# De acuerdo a la literatura revisada, las complicaciones son muy comunes (se comprueba con el 55% observado en la en conjunto de datos) y que éstas pueden ser muy leves.

# Para efectos del presente trabajo, nos concentramremos en predecir si puede haber complicaciones letales, que son los casos más importantes.